In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [2]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from skforecast.metrics import mean_absolute_scaled_error

## Single series

In [3]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0},
    verbose=False
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

# Train-validation dates
# ==============================================================================
end_train = '2002-01-01 23:59:00'

In [4]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = False,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True  
                      )

print(f"Backtesting MASE: {metric}")

# Manual check
# ==============================================================================
mae_foreast = mean_absolute_error(data.loc[end_train:], predictions)
naive_in_sample_foreast = data.loc[:end_train].shift(1).dropna()
mae_in_sample = mean_absolute_error(data.loc[naive_in_sample_foreast.index], naive_in_sample_foreast)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")
assert metric == mae_foreast/mae_in_sample

  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting MASE: 2.344471182248395
mae forecast: 0.19922359134545478
mae in sample: 0.08497591817460318
mase: 2.344471182248395


In [5]:
y_true = data.loc[end_train:]
y_pred = predictions['pred']
y_train = data.loc[:end_train]
mean_absolute_scaled_error(y_true, y_pred, y_train)

2.344471182248395

In [6]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = False,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = 3,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True  
                      )

print(f"Backtesting MASE: {metric}")

# Manual check
# ==============================================================================
refit_intervals = [
    ('1991-07-01 00:00:00', '2002-01-01 00:00:00'),
    ('1991-07-01 00:00:00', '2004-07-01 00:00:00'),
    ('1991-07-01 00:00:00', '2007-01-01 00:00:00')
]

mae_foreast = mean_absolute_error(data.loc[end_train:], predictions)
y_train = pd.concat([data.loc[start:end] for start, end in refit_intervals])
y_train = y_train.loc[~y_train.index.duplicated(keep='first')]
naive_in_sample_foreast = y_train.shift(1).dropna()
mae_in_sample = mean_absolute_error(y_train.loc[naive_in_sample_foreast.index], naive_in_sample_foreast)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")
assert metric == mae_foreast/mae_in_sample

Information of backtesting process
----------------------------------
Number of observations used for initial training: 127
Number of observations used for backtesting: 77
    Number of folds: 8
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 7 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-02-01 00:00:00 -- 2002-11-01 00:00:00  (n=10)
Fold: 1
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-12-01 00:00:00 -- 2003-09-01 00:00:00  (n=10)
Fold: 2
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2003-10-01 00:00:00 -- 2004-07-01 00:00:00  (n=10)
Fold: 3
    Training:   1991-07-01 00:00:00 -- 2004-07-01 00:00:00  (n=157)
    Validation: 2004-08-01 00:00:00 -- 2005-05-01 00:00:00  (n=10)
Fold: 4
    Training:   1991-07-01 00:00:00 -- 2004-07-01 00:00:00  (n=1

  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting MASE: 2.462755601999461
mae forecast: 0.23185816123376632
mae in sample: 0.09414582634408604
mase: 2.462755601999461


## Multiseries

In [7]:
# Download data
# ==============================================================================
data = fetch_dataset(name="items_sales")
data.head()

# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [8]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = True,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True  
                      )

print(f"Backtesting MASE: {metric}")

# Manual check
# ==============================================================================
mae_foreast = mean_absolute_error(
    data.loc[end_train:], predictions, multioutput="raw_values"
)
naive_in_sample_foreast = data.loc[:end_train].shift(1).dropna().drop_duplicates()
mae_in_sample = mean_absolute_error(
    data.loc[naive_in_sample_foreast.index],
    naive_in_sample_foreast,
    multioutput="raw_values",
)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")
assert (metric['mean_absolute_scaled_error'] == mae_foreast/mae_in_sample).all()

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


  0%|          | 0/17 [00:00<?, ?it/s]

Backtesting MASE:    levels  mean_absolute_scaled_error
0  item_1                    1.091527
1  item_2                    1.563339
2  item_3                    0.959262
mae forecast: [1.69131952 3.70767238 3.57171715]
mae in sample: [1.54949824 2.37163672 3.72340161]
mase: [1.09152723 1.56333908 0.95926186]


In [9]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = True,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = 10,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True  
                      )

print(f"Backtesting MASE: {metric}")

# Manual check
# ==============================================================================
# Manual check
refit_intervals = [
    ('2012-01-01 00:00:00', '2014-07-15 00:00:00'),
    ('2012-04-10 00:00:00', '2014-10-23 00:00:00')
]

mae_foreast = mean_absolute_error(
    data.loc[end_train:], predictions, multioutput="raw_values"
)
data_train = pd.concat([data.loc[start:end, :] for start, end in refit_intervals])
data_train = data_train.loc[~data_train.index.duplicated(keep='first'), :]
naive_in_sample_foreast = data_train.shift(1).dropna()
mae_in_sample = mean_absolute_error(
    data.loc[naive_in_sample_foreast.index],
    naive_in_sample_foreast,
    multioutput="raw_values",
)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")
assert (metric['mean_absolute_scaled_error'] == mae_foreast/mae_in_sample).all()

Information of backtesting process
----------------------------------
Number of observations used for initial training: 927
Number of observations used for backtesting: 170
    Number of folds: 17
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-07-16 00:00:00 -- 2014-07-25 00:00:00  (n=10)
Fold: 1
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-07-26 00:00:00 -- 2014-08-04 00:00:00  (n=10)
Fold: 2
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-08-05 00:00:00 -- 2014-08-14 00:00:00  (n=10)
Fold: 3
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-08-15 00:00:00 -- 2014-08-24 00:00:00  (n=10)
Fold: 4
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-08-25 00:00:00 --

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


  0%|          | 0/17 [00:00<?, ?it/s]

Backtesting MASE:    levels  mean_absolute_scaled_error
0  item_1                    1.106355
1  item_2                    1.571983
2  item_3                    1.005813
mae forecast: [1.69056888 3.67036308 3.71593437]
mae in sample: [1.52805244 2.33486251 3.69445992]
mase: [1.10635527 1.57198253 1.00581261]
